In [ ]:
#addprocs()
using SortSpikes, ExtractSpikes, DistributedArrays, Gtk.ShortNames, Winston

In [ ]:
#pyqtgraph testing
using PySide
reload(Pkg.dir("PySide", "src", "pyqtgraph.jl"))
using PyQtGraph
using PyCall

In [ ]:
#make data structure for spike sorting
st=Array(Sorting,64)

#In order to have spike waveforms updated as fast as possible
#you would ideally have some sort of read only pointer that looks
#at the waveforms location in the distributed array. Unfortunately, I 
#can't find a straightforward way to do this. The solution I came up with is to have the 
#plot and waveform both reference to a shared array for each waveform.

for i=1:64
    st[i]=Sorting(SpikeDetection(),Cluster(),convert(Array{Int64,1},rand(Uint16,600)),zeros(Int,500),zeros(Int,500),2,[convert(SharedArray,zeros(Int64,50)) for j=1:10])
end

s=distribute(st);


In [ ]:
win=@Window(title="myGui")

g=@Grid()

push!(win,g)

c=[@Canvas(200,200) for i=1:32] #Would be nice to adjust

for i=1:4
    for j=1:8
        g[j,i]=c[sub2ind((8,4),j,i)]
    end 
end

showall(win)

f=Array(Any,32)
cu=[Curve(1:50,1:50, color="red") for i=1:32, j=1:10]

p=[FramedPlot() for i=1:32]

for i=1:32
    f[i]=Winston.Figure(c[i],p[i])
    Winston.addfig(Winston._display,i,f[i])
    for j=1:10
        add(p[i],cu[i,j])
    end
    Winston.display(c[i],p[i])
end

#p[1].content1.components[1].attr[:style][:linecolor]="blue"

In [ ]:
waves=[convert(SharedArray,ones(Int64,50)) for i=1:32, j=1:10]

for i=1:32
    for j=1:10
        p[i].content1.components[j].y=waves[i,j]
    end
end

#make the electrodes you want to be on the screen also point to waves
@parallel for i=1:32
    for j=1:10
        s[i].waveforms[j]=waves[i,j]
    end
end

In [ ]:
#Create Window
w = Widget()

set_size(w, 800, 600)

#shows window
raise(w)
# Graphics Layout Widget make grid of plot devices
win = GraphicsLayoutWidget(w)
lyt = GridLayout(w)
addWidget(lyt, win)

# addPlot adds a plot device in each grid element and returns a PyObject
# PyQtGraph.GraphicsPlot object
p = PyObject[win[:addPlot](i,j) for i=1:4, j=1:8];
p = vec(p)

#stores all of the curves that can be present on the plot
#k dictates how many curves can be present at once
#this needs to be shared to be accessed by multiple processes at once
curve = PyObject[p[i][:plot](pen="y") for i=1:32, k=1:10];
for i=1:10, j=1:32
    curve[j,i][:setData](convert(Array{Int64,1},rand(Uint16,50)))
end

## stop auto-scaling
for i=1:32
    p[i][:enableAutoRange]("xy", false) 
end

for i=1:10, j=1:32
    curve[j,i][:setData](ones(Int,50))
end

In [ ]:
waves=[convert(SharedArray,ones(Int64,50)) for i=1:32, j=1:10]

for i=1:32
    for j=1:10
        #curve[i,j][:yData]=waves[i,j]
        curve[i,j][:yDisp]=waves[i,j]
        #curve[i,j][:curve][:yData]=waves[i,j]
    end
end

#make the electrodes you want to be on the screen also point to waves
@parallel for i=1:32
    for j=1:10
        s[i].waveforms[j]=waves[i,j]
    end
end

In [ ]:
#calibrate on first batch of data
#probably want to have a calibration period since 600 samples isn't enough
@parallel for i=1:64
    #s[i].rawSignal=convert(Array{Int64,1},rand(Uint16,600))
    s[i].rawSignal[:]=ones(Int64,600)
    s[i].rawSignal[100]=10000
end

@time map!(onlineCal, s);

In [ ]:
#generate new fake data
for i=1:64
    #s[i].rawSignal=convert(Array{Int64,1},rand(Uint16,600))
    s[i].rawSignal[:]=ones(Int64,600)
    s[i].rawSignal[100]=10000
end

#sort new data
@time map!(onlineSort, s);

In [ ]:
@time curve[1,2][:updateItems]()

In [ ]:
#Wavelet Experiments

function cwt(x::Array{Float64,1},t::Int,psi::Array{Float64,1},bs::Array{Float64,1})

    bs[1]=bs[2]-x[t-20]*psi[1]+x[t]*psi[21]

    for i=2:19
        bs[i]=bs[i+1]-x[t-20]*psi[i]+x[t]*psi[22-i] 
    end

    bs[20]=0.0
    for i=1:20
        bs[20]+=x[t-20+i]*psi[21-i]
    end
    
    return bs
end


function mycwt(x::Array{Float64,1},t::Int, psi::Array{Float64,1})
    
    bs=0.0
    
    for i=1:20
        bs+=x[t-20+i]*psi[21-i]
    end
    
end

function main()
    for i=1:20000
        mycwt(x,t,psi)
    end
end

psi=[0.00021208083980379827
0.00035858968789573785
-0.0021782363581090178
-0.004159358781386048
0.010131117519849788
0.023408156785839195
-0.02816802897093635
-0.09192001055969624
0.05204316317624377
0.4215662066908515
-0.7742896036529562
0.4379916261718371
0.06203596396290357
-0.10557420870333893
-0.0412892087501817
0.03268357426711183
0.01976177894257264
-0.009164231162481846
-0.006764185448053083
0.0024333732126576722
0.0016628637020130838
-0.0006381313430451114
-0.00030225958181306315
0.00014054114970203437
4.134043227251251e-05
-2.1315026809955787e-05
-3.7346551751414047e-06
2.0637618513646814e-06
1.6744288576823017e-07
-9.517657273819165e-08
];

bs=zeros(Float64,20)
    
x=rand(100)
t=50

for j=1:20
    for i=1:20
        bs[j]+=x[t-20+i]*psi[21-i]
    end
end